## 0 Prerequisites provided in labs

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Uninstall packages from Kaggle base image that are not needed.
!pip uninstall -qy jupyterlab jupyterlab-lsp
# Install the google-genai SDK for this codelab.
!pip install -qU 'google-genai==1.7.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.7 MB/s eta 0:00:00


In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

# genai.__version__

### Set API Keys

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

## Data for Analysis
Building the Women's Slolom Data Set without AI Agent Agent Assitance

In [5]:
# Data for Women's Slalom Top 20 - Sun Valley 2025
Wdata = [
    ["Mikaela Shiffrin", "USA", 52.00, 53.92, 105.92],
    ["Lena Dürr", "GER", 52.50, 54.55, 107.05],
    ["Andreja Slokar", "SLO", 52.80, 54.50, 107.30],
    ["Katharina Liensberger", "AUT", 53.00, 54.60, 107.60],
    ["Camille Rast", "SUI", 53.20, 54.70, 107.90],
    ["Wendy Holdener", "SUI", 53.40, 54.80, 108.20],
    ["Anna Swenn-Larsson", "SWE", 53.60, 54.90, 108.50],
    ["Michelle Gisin", "SUI", 53.80, 55.00, 108.80],
    ["Sara Hector", "SWE", 54.00, 55.10, 109.10],
    ["Zrinka Ljutić", "CRO", 54.20, 55.20, 109.40],
    ["Katharina Truppe", "AUT", 54.40, 55.30, 109.70],
    ["Katharina Huber", "AUT", 54.60, 55.40, 110.00],
    ["Mina Fürst Holtmann", "NOR", 54.80, 55.50, 110.30],
    ["Cornelia Öhlund", "SWE", 55.00, 55.60, 110.60],
    ["Emma Aicher", "GER", 55.20, 55.70, 110.90],
    ["Laurence St-Germain", "CAN", 55.40, 55.80, 111.20],
    ["Marie Lamure", "FRA", 55.60, 55.90, 111.50],
    ["Neja Dvornik", "SLO", 55.80, 56.00, 111.80],
    ["Hanna Aronsson Elfman", "SWE", 56.00, 56.10, 112.10],
    ["Ali Nullmeyer", "CAN", 56.20, 56.30, 112.50],
]

## Database
### Build

In [6]:
%load_ext sql
%sql sqlite:///sample.db

In [7]:
%%sql
CREATE TABLE IF NOT EXISTS WomenSL (
    wsl_Id INTEGER PRIMARY KEY AUTOINCREMENT
    ,Skier_NME VARCHAR(100) NOT NULL
    ,Country_CD VARCHAR(3) NOT NULL
    ,Run1 DECIMAL(3,4) NOT NULL
    ,Run2 DECIMAL(3,4) NOT NULL
    ,TotalTime DECIMAL(3,4) NOT NULL
);

CREATE TABLE IF NOT EXISTS MenSL (
    msl_Id INTEGER PRIMARY KEY AUTOINCREMENT
    ,Skier_NME VARCHAR(100) NOT NULL
    ,Country_CD VARCHAR(3) NOT NULL
    ,Run1 DECIMAL(3,4) NOT NULL
    ,Run2 DECIMAL(3,4) NOT NULL
    ,TotalTime DECIMAL(3,4) NOT NULL
);

 * sqlite:///sample.db
Done.
Done.


[]

### Explicitly populate database
this explicit process will be tested to see if AI Agent can find the corresponding data from the Men's race and insert it into the appropriate table 

In [8]:
import sqlite3
conn = sqlite3.connect("sample.db")
cursor = conn.cursor()

cursor.executemany("""
    INSERT INTO WomenSL (Skier_Nme, Country_CD, Run1, Run2, TotalTime)
    VALUES (?, ?, ? ,? ,?)
""", Wdata)

## Build db Tools for Agent to use 
### Add functions so engine can examine and perform database operations
[Modified from Day 3 - Function calling with the Gemini API](https://www.kaggle.com/code/markishere/day-3-function-calling-with-the-gemini-api) Copyright 2025 Google LLC.

In [10]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()
    
## pull list of tables to identify where to permorm DML
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables()')

    cursor = conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]

## Describe Tables for DML Ops
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(f' - DB CALL: describe_table({table_name})')

    cursor = conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]

def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()
# list of db tools for AI 
db_tools = [list_tables, describe_table, execute_query]

## V. Data for Grounding 2: AI Automated insert into Men's Slalom Results
Unlike 'manually' building the data set for the women's results, this expirement trys to have the Agent acquire the data and insert it into the correct table

In [14]:
instruction = """You are a helpful chatbot that aids the user in performing statistical analysis of ski race results which are stored and you can update ina 
SQL database  You will take the users questions, search for the relevant data online, store them in SQL Databases and translate question into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see what tables are present, describe_table to understand the
schema, and execute_query to issue an SQL SELECT query.

Use descriptive and inferential statistics selecting the most appropriate method based on user's question
assume race results are a population and not a sample

"""

client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

### Retry Policy (if needed)

In [13]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)